In [ ]:
# in this version, i will combine output of CNN regression models with the demographic data
# CNN model I am currently using is:
# "elementary_227_227_zoom_16_regression_2022_12_31_hidden_32_relu.h5"
# from Transfer_Learning_final_project_06
# Xception with 128 batch size, 100 epochs, 32 hidden tanh units
# training MSE = 0.1398
# validation MSE = 0.1423
# testing R = 0.4257, R^2 = 0.1812


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential, load_model
import keras_tuner
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fitter import Fitter, get_common_distributions, get_distributions
import random

In [44]:
# load csv
Data = pd.read_csv("C:\\Users\\Joe\\Box\\Wesley\\Learning Analytics\\HUDK 4050\\Final\\Data\\Data_Table_11_elementary.csv")
N = np.shape(Data)[0]

In [11]:
# load images
folderpath = "C:\\Users\\Joe\\Box\\Wesley\\Learning Analytics\\HUDK 4050\\Final\\Data\\google_maps_satellite_227_227_16_elementary\\"
img_height = 227
img_width = 227
Images = np.zeros(shape = (N, img_height, img_width, 3))
TestScores = Data.cs_mn_avg_ol
for i in range(N):
    id = Data.school_code[i]
    im = tf.keras.utils.load_img(folderpath + "school_" + str(id) + ".jpg",
                                 grayscale=False, color_mode="rgb")
    Images[i,:,:,:] = im

In [12]:
# load training, validation, testing indices
data = np.load('elementary_227_227_zoom_16_regression_2022_12_31_hidden_32_relu_variables.npz')
ind = data['ind']
ind_train = data['ind_train']
ind_val = data['ind_val']
ind_test = data['ind_test']
mse_train = data['mse_train']
mse_val = data['mse_val']
del data

x_train = Images[ind_train,:,:,:]
y_train = TestScores[ind_train]
x_val = Images[ind_val,:,:,:]
y_val = TestScores[ind_val]
x_test = Images[ind_test,:,:,:]
y_test = TestScores[ind_test]

print(N)
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_val))
print(np.shape(y_val))
print(np.shape(x_test))
print(np.shape(y_test))
print(ind[0:5]) # should be [14690  1444   351 18769  5916]

18939
(14204, 227, 227, 3)
(14204,)
(2841, 227, 227, 3)
(2841,)
(1894, 227, 227, 3)
(1894,)
[14690  1444   351 18769  5916]


In [16]:
# load model
model = load_model('elementary_227_227_zoom_16_regression_2022_12_31_hidden_32_relu.h5')

In [24]:
print(model.summary(0))

for layer in model.layers:
    print(layer.name)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 227, 227, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 227, 227, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 227, 227, 3)      0         
 a)                                                              
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [2]:
# Get model predictions along with hidden layer outputs
layer_name = 'dense'
num_hidden = 32
hidden_layer_model = keras.Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

# training predictions
y_train_pred = np.zeros(shape = (np.shape(x_train)[0]))
h_train_pred = np.zeros(shape = (np.shape(x_train)[0],num_hidden))
for i in range(np.shape(x_train)[0]):
    im = x_train[i,:,:,:]
    im = im[np.newaxis,:,:,:]
    y_train_pred[i] = model.predict(im)
    h_train_pred[i,:] = hidden_layer_model.predict(im)
    
# validation predictions
y_val_pred = np.zeros(shape = (np.shape(x_val)[0]))
h_val_pred = np.zeros(shape = (np.shape(x_val)[0],num_hidden))
for i in range(np.shape(x_val)[0]):
    im = x_val[i,:,:,:]
    im = im[np.newaxis,:,:,:]
    y_val_pred[i] = model.predict(im)
    h_val_pred[i,:] = hidden_layer_model.predict(im)
    
# testing predictions
y_test_pred = np.zeros(shape = (np.shape(x_test)[0]))
h_test_pred = np.zeros(shape = (np.shape(x_test)[0],num_hidden))
for i in range(np.shape(x_test)[0]):
    im = x_test[i,:,:,:]
    im = im[np.newaxis,:,:,:]
    y_test_pred[i] = model.predict(im)
    h_test_pred[i,:] = hidden_layer_model.predict(im)

NameError: name 'keras' is not defined

In [43]:
# saving predictions because they took a really long time
np.savez('Transfer_Learning_Combined_with_Data_Table_01_predictions.npz',y_train_pred=y_train_pred,h_train_pred=h_train_pred,y_val_pred=y_val_pred,h_val_pred=h_val_pred,y_test_pred=y_test_pred,h_test_pred=h_test_pred)

In [72]:
# var = Data.student_teacher_ratio
# var = Data.perwht
# var = Data.ipr_est

nanind = np.isnan(var)

R = np.corrcoef(TestScores[~nanind], var[~nanind])
print(R)
print(R[0,1]*R[0,1])

[[1.         0.67474531]
 [0.67474531 1.        ]]
0.45528122869616394


In [1]:
var1 = Data.perwht[ind_test]
var2 = Data.cs_mn_avg_ol[ind_test]

nanind = np.logical_or(np.isnan(var1),np.isnan(var2))

R = np.corrcoef(var1[~nanind], var2[~nanind])
print(R)
print(R[0,1]*R[0,1])

NameError: name 'Data' is not defined

In [ ]:
R = np.corrcoef(y_test, y_pred)
print(R)
print(R[0,1]*R[0,1])

plt.plot(y_test, y_pred, 'o', color='black');
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')